<a href="https://colab.research.google.com/github/ralik45/Housing_Price/blob/main/housing_price_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'melbourne-housing-market:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F423%2F126865%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240810%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240810T092018Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D2731f70048258ddfa7215582e23a9055f38488cb8485a078fc621e09112f1ff21bd997554e3ebd2746cb89bc1e592633c80ab6c5cb4a200c171fccefef217098b3422e91a66a80d4478ed5e7d990ceb9a7f84daefd9f8f038fefb80e7c33b9b015867569356ac72bf44f5fe4179163f3bf31b528fdc2ae626bcb79aaebfbebf372245802e421fb8eaf725b04c3e733f0d7a7fa6b951272c2bd1ea6ad346430a0c8245685a8024ba43f06f27eef55665cdc2e1f378538d4841449276128f6439dc7880660cbeec9348d4fc379c75738629d069eb8396677beb2151ce91f8a10fd96f1035eca83d308b0d6f8145a4cd26115021ae3a5396a07b4f10a65d60d9b58'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from dateutil.parser import parse
from datetime import datetime
from scipy.stats import norm

# import all what you need for machine learning
import sklearn
from sklearn.model_selection import cross_val_predict
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_validate
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import RobustScaler

colors = ["#bfd3e6", "#9b5b4f", "#4e4151", "#dbba78", "#bb9c55", "#909195","#dc1e1e","#a02933","#716807","#717cb4"]

In [ ]:
house = pd.read_csv("/kaggle/input/melbourne-housing-market/MELBOURNE_HOUSE_PRICES_LESS.csv")
house

In [ ]:
house.describe().T.style.bar(subset=['mean'],color=colors[3]).background_gradient(subset=['std','50%','max'])

In [ ]:
sns.distplot(house['Price'],color=colors[7])
plt.axvline(x=house['Price'].mean(), color=colors[7], linestyle='--', linewidth=2)
plt.title('Sales');

In [ ]:
missing = house.isnull().sum()
missing = missing[missing > 0]
missing = missing.sort_values(ascending = False)
missing.plot.bar(color=colors[7])
plt.title('Missing Data');

In [ ]:
no_missing = house[house['Price'].notnull()]
no_missing.isnull().sum().sort_values(ascending=False)

In [ ]:
no_missing

In [ ]:
no_missing['Type'].unique()

In [ ]:
no_missing = no_missing.drop(['Postcode','Propertycount','Address','Suburb','SellerG','Method', 'CouncilArea', 'Date'],axis=1)

In [ ]:
no_missing.head()

In [ ]:
df = no_missing
df.info()

In [ ]:
hD= pd.get_dummies(df, columns= ['Type', 'Regionname'])
hD.info()

In [ ]:
hD

In [ ]:
import numpy as np
import cufflinks as cf
import plotly.graph_objs as go
from plotly.offline import iplot

# Konversi array NumPy menjadi DataFrame
df_hD = pd.DataFrame(hD)

# Hitung korelasi
df_corr = df_hD.corr()

# Buat heatmap
trace = [go.Heatmap(
    z=df_corr.values.tolist(),
    x=df_corr.columns,
    y=df_corr.columns,
    colorscale='bluered'
)]

layout = go.Layout(
    title="Heatmap <br> Detecting Correlations"
)

fig = dict(data=trace, layout=layout)

# Plot heatmap
iplot(fig, filename='pandas-heatmap')


In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [ ]:
model = tf.keras.Sequential([
    keras.layers.Dense(units=32, input_shape=[1], activation='linear'),
    keras.layers.Dense(units=1, activation='linear')
])
model.compile(optimizer='sgd', loss='mean_squared_error')
model.fit(X, Y, epochs=1000)
print(model.predict([-2.0, 10.0]))
return model

In [ ]:
#create x and y variables
X = hD.drop("Price", axis=1)
Y = hD["Price"].copy()
#transform to array size
#feature scaling
scaler = RobustScaler()
hD= scaler.fit_transform(hD.astype(np.float64))

In [ ]:
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = .20, random_state= 42)

In [ ]:
#fit decision tree
tree = DecisionTreeRegressor()
tree.fit(x_train, y_train)
#fit random forest
forest = RandomForestRegressor(n_jobs=-1)
forest.fit(x_train, y_train)
#fit regression
lin_reg = LinearRegression(n_jobs=-1)
lin_reg.fit(x_train, y_train)

In [ ]:
models= [('lin_reg', lin_reg), ('random forest', forest), ('decision tree', tree)]
from sklearn.metrics import mean_squared_error
for i, model in models:
    predictions = model.predict(x_train)
    MSE = mean_squared_error(y_train, predictions)
    RMSE = np.sqrt(MSE)
    msg = "%s = %.2f" % (i, round(RMSE, 2))
    print('RMSE of', msg)

In [ ]:
for i, model in models:
    # Make predictions on train data
    predictions = model.predict(x_train)
    # Performance metrics
    errors = abs(predictions - y_train)
    # Calculate mean absolute percentage error (MAPE)
    mape = np.mean(100 * (errors / y_train))
    # Calculate and display accuracy
    accuracy = 100 - mape
    #print result
    msg = "%s= %.2f"% (i, round(accuracy, 2))
    print('Accuracy of', msg,'%')

In [ ]:
test_data = pd.read_csv('/kaggle/input/melbourne-housing-market/Melbourne_housing_FULL.csv')

In [ ]:
#create x and y variables
X = test_data.drop("Price", axis=1)
Y = test_data["Price"].copy()
#transform to array size
#feature scaling
scaler = RobustScaler()
test_data= scaler.fit_transform(hD.astype(np.float64))

In [ ]:
for i, model in models:
    # Make predictions on train data
    predictions = model.predict(x_train)
    # Performance metrics
    errors = abs(predictions - y_train)
    # Calculate mean absolute percentage error (MAPE)
    mape = np.mean(100 * (errors / y_train))
    # Calculate and display accuracy
    accuracy = 100 - mape
    #print result
    msg = "%s= %.2f"% (i, round(accuracy, 2))
    print('Accuracy of', msg,'%')

In [ ]:
models= [('lin_reg', lin_reg), ('forest', forest), ('dt', tree)]
scoring = ['neg_mean_squared_error', 'neg_mean_absolute_error', 'r2']

#for each model I want to test three different scoring metrics. Therefore, results[0] will be lin_reg x MSE,
# results[1] lin_reg x MSE and so on until results [8], where we stored dt x r2

results= []
metric= []
for name, model in models:
    for i in scoring:
        scores = cross_validate(model, x_train, y_train, scoring=i, cv=10, return_train_score=True)
        results.append(scores)

In [ ]:
#this is an example of the stored results
results[8]